## **Financial Default Prediction - Credit Risk**
**Problem statement :** Building a Financial Defult Prediction - Credit Risk Project based on Data from Kaggle that has person Financial Information and past Loan Information along with Demographic information.


### **Project Methodology**

- This Project using the Kaggle Data of Loan Default Prediction.
- Using Python, that load data and then pre-processed and saved in CSV File.
- Loading that same CSV file to insert into Vector DB using Embedding Model from Hugging Face.
- Building RAG QA Chain using Langchain and building the RAG architecture using Zypher 7B LLM (Open Source).
- Checking the Response if it will Default or not.

In [2]:
import pandas as pd
import numpy as np

df= pd.read_csv('/content/Loan_default.csv')
df.head()

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


In [3]:
# Checking for missing values and handle them

print(df.isnull().sum())

LoanID            0
Age               0
Income            0
LoanAmount        0
CreditScore       0
MonthsEmployed    0
NumCreditLines    0
InterestRate      0
LoanTerm          0
DTIRatio          0
Education         0
EmploymentType    0
MaritalStatus     0
HasMortgage       0
HasDependents     0
LoanPurpose       0
HasCoSigner       0
Default           0
dtype: int64


In [6]:
def create_prompt(data_point) :
     prompt = """
       Predict the default probability for the following loan applicant:

    - Age: {data_point['Age']}
    - Income: {data_point['Income']}
    - Loan Amount: {data_point['LoanAmount']}
    - Credit Score: {data_point['CreditScore']}
    - Months Employed: {data_point['MonthsEmployed']}
    - Number of Credit Lines: {data_point['NumCreditLines']}
    - Interest Rate: {data_point['InterestRate']}
    - Loan Term: {data_point['LoanTerm']}
    - Debt-to-Income Ratio: {data_point['DTIRatio']}
    - Education: {data_point['Education']}
    - Employment Type: {data_point['EmploymentType']}
    - Marital Status: {data_point['MaritalStatus']}
    - Has Mortgage: {data_point['HasMortgage']}
    - Has Dependents: {data_point['HasDependents']}
    - Loan Purpose: {data_point['LoanPurpose']}
    - Has Co-Signer: {data_point['HasCoSigner']}
    """
     return prompt

In [7]:
#  Create a list to store prompts and response
data_lists= []

In [8]:
# generate prompts and responses

for index , row in df.iterrows() :
  prompt = create_prompt(row)
  response = row['Default']
  data_lists.append((prompt , response))

In [10]:
data_lists[0]

(" \n       Predict the default probability for the following loan applicant:\n\n    - Age: {data_point['Age']}\n    - Income: {data_point['Income']}\n    - Loan Amount: {data_point['LoanAmount']}\n    - Credit Score: {data_point['CreditScore']}\n    - Months Employed: {data_point['MonthsEmployed']}\n    - Number of Credit Lines: {data_point['NumCreditLines']}\n    - Interest Rate: {data_point['InterestRate']}\n    - Loan Term: {data_point['LoanTerm']}\n    - Debt-to-Income Ratio: {data_point['DTIRatio']}\n    - Education: {data_point['Education']}\n    - Employment Type: {data_point['EmploymentType']}\n    - Marital Status: {data_point['MaritalStatus']}\n    - Has Mortgage: {data_point['HasMortgage']}\n    - Has Dependents: {data_point['HasDependents']}\n    - Loan Purpose: {data_point['LoanPurpose']}\n    - Has Co-Signer: {data_point['HasCoSigner']}\n    ",
 0)

In [ ]:
!pip install langchain langchain-community

In [13]:
from langchain.docstore.document import Document

In [19]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [22]:
!pip install sentence-transformers

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
hf  = HuggingFaceEmbeddings()

In [ ]:
!pip install chromadb

In [25]:
persist_directory = 'db'

In [ ]:
from langchain.vectorstores import Chroma
data_lists = [Document(page_content=t[0], Defult=t[1]) for t in data_lists]
langchain_chroma = Chroma.from_documents(
    documents=data_lists,
    collection_name="default_prediction",
    embedding=hf,
    persist_directory=persist_directory
)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub



In [ ]:
template = """
 You are an Credit Risk Expert in Financial Text data , analyse them and predict is the Given question :
 {question}
 Context : {context}

Answer :


"""

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])


In [ ]:
model_id = 'HuggingFaceH4/zephyr-7b-beta'
llm =  HuggingFaceHub(repo_id=model_id,
                      model_kwargs={"temperature":0.5,
                                    "max_length":512})

In [ ]:
retriever = langchain_chroma.as_retriever()
qa_chain =  RetrievalQA.from_chain_type(
    llm = llm ,
    retriever = retriever,
    chain_type_kwargs = {"prompt":prompt}
)



In [ ]:
# question = "The company reported inflated revenues by including sales that never occurred."
question = "\n    Predict the default probability for the following loan applicant:\n\n    - Age: 56\n    - Income: 85994\n    - Loan Amount: 50587\n    - Credit Score: 520\n    - Months Employed: 80\n    - Number of Credit Lines: 4\n    - Interest Rate: 15.23\n    - Loan Term: 36\n    - Debt-to-Income Ratio: 0.44\n    - Education: Bachelor's\n    - Employment Type: Full-time\n    - Marital Status: Divorced\n    - Has Mortgage: Yes\n    - Has Dependents: Yes\n    - Loan Purpose: Other\n    - Has Co-Signer: Yes\n    "
# question = "Revenue was recognized prematurely before the actual sales occurred."
# question = "The balance sheet provides a true and fair view of the company’s financial position."


In [ ]:
ff = qa_chain({"query": question})
    display(ff["result"])